# Transfer learning in Keras

In this notebook, we'll cover how to load a pre-trained model (in this case, VGGNet19) and finetune it for a new task.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


#### Load the pre-trained VGG19 model

In [4]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Freeze the pre-trained model

In [79]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [88]:
model = Sequential()
model.add(vgg19)
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Compile the model for training

In [90]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

The dataset is available for download [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home). You should download the zipfile and extract the contents into a folder called `hot-dog-not-hot-dog` in the main `notebooks` directory.

In [91]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   data_format='channels_last',
                                   rotation_range=30,
                                   horizontal_flip=True,
                                   fill_mode='reflect')

test_datagen = ImageDataGenerator(rescale=1.0/255,
                                  data_format='channels_last')

In [92]:
batch_size=32

In [93]:
train_generator = train_datagen.flow_from_directory(directory='./hot-dog-not-hot-dog/train',
                                                     target_size=(224, 224),
                                                     classes=['hot_dog','not_hot_dog'],
                                                     class_mode='categorical',
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     seed=42)

test_generator = test_datagen.flow_from_directory(directory='./hot-dog-not-hot-dog/test',
                                                     target_size=(224, 224),
                                                     classes=['hot_dog','not_hot_dog'],
                                                     class_mode='categorical',
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [94]:
model.fit_generator(train_generator, steps_per_epoch=15, epochs=40, validation_data=test_generator, validation_steps=15)

Epoch 1/40
15/15 [==============================] - 6s - loss: 1.2048 - acc: 0.5312 - val_loss: 0.5210 - val_acc: 0.7438
Epoch 2/40
15/15 [==============================] - 4s - loss: 0.6471 - acc: 0.6810 - val_loss: 0.4636 - val_acc: 0.7778
Epoch 3/40
15/15 [==============================] - 4s - loss: 0.5018 - acc: 0.7565 - val_loss: 0.4323 - val_acc: 0.7970
Epoch 4/40
15/15 [==============================] - 4s - loss: 0.4211 - acc: 0.8093 - val_loss: 0.4886 - val_acc: 0.7607
Epoch 5/40
15/15 [==============================] - 4s - loss: 0.4173 - acc: 0.8290 - val_loss: 0.4889 - val_acc: 0.7585
Epoch 6/40
15/15 [==============================] - 4s - loss: 0.3307 - acc: 0.8493 - val_loss: 0.4242 - val_acc: 0.8034
Epoch 7/40
15/15 [==============================] - 4s - loss: 0.3520 - acc: 0.8264 - val_loss: 0.5458 - val_acc: 0.7286
Epoch 8/40
15/15 [==============================] - 4s - loss: 0.2988 - acc: 0.8660 - val_loss: 0.4453 - val_acc: 0.7875
Epoch 9/40
15/15 [==============